In [6]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from io import BytesIO  
%matplotlib inline

In [35]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES]; CONNECTED_PART_INDICES

[(11, 5),
 (7, 5),
 (7, 9),
 (11, 13),
 (13, 15),
 (12, 6),
 (8, 6),
 (8, 10),
 (12, 14),
 (14, 16),
 (5, 6),
 (11, 12)]

In [39]:
def getBase64Image(filePath):
    with open(filePath, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        my_string = my_string.decode('utf-8')
    return my_string
    

In [40]:
def convertToBase64(image):
    retval, buffer = cv2.imencode('.jpg', image)
    my_string = base64.b64encode(buffer)
    my_string = my_string.decode('utf-8')
    return my_string
    
  

In [48]:
def get_adjacent_keypoints(keypoints):
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            
        

In [38]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')


In [42]:
#Sending data to server
def getPoses(image_string):
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+image_string}
    r = req.post(url=url2 , data = data)
    poses = r.json()
    return poses

In [232]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 
    

In [230]:
def draw_skelton(poses,img):
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [64]:
def draw_skel_and_kp(poses , img, color):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=color,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=list(color))
    return out_img
    
    
        

In [107]:
def mapIntToColours(fraction,totalColors):
    viridis = cm.get_cmap('magma', totalColors)
    rgba = viridis(fraction)
    bgr = ( int((round(rgba[2] *255))) , int((round(rgba[1] *255))) , int((round(rgba[1] *255))) )
    return bgr
    

In [86]:
mapIntToColours((1/58),58)

(90, 7, 7)

In [49]:

img = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/Screenshot 2020-01-28 at 7.10.58 AM.png')
myString =convertToBase64(img)
poses = getPoses(myString)
out_img = draw_skel_and_kp(poses,img)
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/shootingPoset.jpg' ,out_img) 

True

In [50]:
img.dtype

dtype('uint8')

In [51]:
np.zeros(img.shape, dtype=np.uint8).shape

(672, 1275, 3)

In [108]:
videoPath ='/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/PosenetUsingCDN.ipynb.mov'
cap = cv2.VideoCapture(videoPath)
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
count =0;
mergedImage = None;
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        if count==0: mergedImage = np.zeros(frame.shape, dtype=np.uint8)
        imageString = convertToBase64(frame)
        poses = getPoses(imageString)
        fraction = ((count + 1) / totalFrames)
        
        color = mapIntToColours(fraction,totalFrames)
        print(color)
        mergedImage = draw_skel_and_kp(poses , mergedImage , color)
        count += 1
    else:
        cap.release()
        break
cap.release()
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/mergedImageBlack.png', mergedImage)   

(9, 1, 1)
(18, 3, 3)
(28, 6, 6)
(36, 9, 9)
(47, 12, 12)
(56, 14, 14)
(68, 16, 16)
(78, 17, 17)
(90, 17, 17)
(99, 17, 17)
(108, 16, 16)
(114, 15, 15)
(120, 16, 16)
(123, 17, 17)
(125, 20, 20)
(127, 22, 22)
(128, 26, 26)
(129, 28, 28)
(129, 31, 31)
(129, 34, 34)
(130, 37, 37)
(129, 39, 39)
(129, 42, 42)
(128, 44, 44)
(127, 46, 46)
(126, 48, 48)
(124, 51, 51)
(123, 53, 53)
(120, 56, 56)
(118, 58, 58)
(118, 58, 58)
(112, 64, 64)
(108, 69, 69)
(105, 72, 72)
(101, 78, 78)
(98, 83, 83)
(95, 90, 90)
(93, 96, 96)
(92, 105, 105)
(92, 112, 112)
(93, 121, 121)
(95, 129, 129)
(98, 138, 138)
(102, 146, 146)
(107, 155, 155)
(111, 163, 163)
(118, 172, 172)
(123, 180, 180)
(130, 189, 189)
(136, 196, 196)
(144, 205, 205)
(151, 213, 213)
(160, 222, 222)
(167, 229, 229)
(176, 238, 238)
(184, 246, 246)
(191, 253, 253)
(191, 253, 253)


True

In [15]:
apple1 = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/apple1.jpg')
apple2 = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/apple2.jpg')


apple1 = cv2.resize(apple1 , (450,450))
apple2 = cv2.resize(apple1 , (450,450))
change = apple1 + apple2 
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/change.jpg' , change)

True

In [136]:
img = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/Screenshot 2020-01-28 at 7.10.58 AM.png')

In [137]:
img= cv2.resize(img , (850,480))

In [138]:
cv2.imwrite('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/smaller.png', img)

True

In [89]:
plasma = cm.get_cmap('hot', 60)
print(viridis)

In [34]:
rgba = plasma(0.5)
(rgba[0] * 255, rgba[1] * 255,rgba[2] * 255)


(31.97547, 146.45109, 140.01693)

In [52]:
list(rbga)

[0.959276, 0.921407, 0.151566, 1.0]

In [74]:
(1/totalFrames)*255

4.396551724137931

In [72]:
count

58